# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 29.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city not just because it's the capital, but also because it's home to many iconic landmarks, such as the White House, the U.S. Capitol, and numerous museums and monuments. If you're interested in American history or politics, learning more about Washington, D.C. could be really fascinating!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  200


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets revolve around various aspects of the upcoming election, with a strong focus on the U.S. political climate. Key themes include concerns about election integrity, with accusations of potential election interference and fraud, particularly involving the Democratic Party. There are discussions about the role of media and misinformation, as well as the influence of dark money in politics. Some tweets highlight specific election-related events, such as court rulings in Georgia and voter registration issues in Pennsylvania. Others mention political figures like Donald Trump, Kamala Harris, and Joe Biden, with debates over their roles and actions. Additionally, there are mentions of election predictions, market reactions, and the importance of voter turnout. Overall, the tweets reflect a highly charged and polarized political environment leading up to the election.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election concerns, misinformation, and predictions, with a focus on Trump, Kamala Harris, and potential election interference.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily focus on election-related discussions, with minimal mention of AI. The conversation revolves around election integrity, misinformation, and political strategies. There is a notable absence of AI-related discourse, as the tweets are dominated by political opinions, election predictions, and concerns about potential election interference.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
@BretBaier This will be the biggest pillow fight of the election process. She was indeed the Border Czar, she lied to us about the health of President Biden, used tax payer $ to pay for a convicted felons sex change,  no wars when Trump was President and what is her position on the freedom… https://t.co/PEgwpFAM5q
===
harassment
RT @caslernoel: Never lose sight of what a clownish little man Mike Johnson is and how he ALREADY worked on behalf of Trump’s attempts at o…
===
violence
RT @democracynow: Uncommitted Co-Founder Abbas Alawieh on U.S. Election &amp; Family in Lebanon Fleeing Israeli Bombs https://t.co/f00eSotc84
===
harassment
RT @maddenifico: Some of you self-righteous motherfuckers on the left are once again overthinking this. It's as if you faux-prog frauds lea…
===
violence
RT @ecomarxi: Biden/Harris [during a full year of genocide &amp; overwhelming domestic &amp; international pressure to embargo Israel]: We will NEV…
===
harassment
@LauraLoomer Their time w

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

@Lucas_FCGB 您可以亲自申请和投递邮寄选票。

https://t.co/cYiPpFttpN
RT @VigilantFox: 关于即将到来的选举有些不对劲，感觉民主党计划窃取选举。

“今天在乔治亚州…
.@dYdX 推出了特朗普永续预测市场 (TRUMPWIN-USD)，您可以在特朗普的选举机会中做多或做空，杠杆高达20倍。提供高级订单类型、专业图表和完全去中心化。别人预测，你来交易;) 👉… https://t.co/JZlC8oKiEP
@CentristMadness 因为没有其他人了解它们的运作方式或真正关心

这就是我们应得的选举和媒体环境
RT @Craptld2kids: 只是提醒一下他们四年前对我们做了什么。当他们在11月5日的选举中失利时，要准备好应对更多的恶作剧。ht…
RT @whstancil: 生活在摇摆州的人们：我们正被右翼虚假信息淹没。所有大型国家媒体都拒绝去…
RT @StephMillerShow: 威斯康星州众议员马克·波坎（D-WI）将在广告后加入我们，讨论选举状况，特别是在关键州威斯康星州的情况……
RT @KamalaHarris: 今天距离选举日还有三周。

让我们撸起袖子，把握每一个时刻。

这将需要……
RT @argosaki: ‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️👇👇👇👇👇👇👇👇👇👇👇👇👇👇👇👇👇👇👁️👁️👁️👁️👁️👁️👁️👇
来了：

2020年大选的亚利桑那州参议院审计…
RT @realTrumpNewsX: 🚨突发新闻：唐纳德·特朗普呼吁将选举日设为全国假日，并实施单日投票和纸质选票…
@Real_RobN 他们信誓旦旦地说这是一场公平的选举。
RT @TristanSnell: 最新消息：乔治亚州法院裁定选举结果认证是强制性的——选举主管和成员……
RT @kangaroos991: 也许这应该每天都发布，直到选举？（你知道该怎么做！）

#特朗普不适合 #卡玛拉竞选总统 💙 https://…
RT @mjfree: 我将每天发布这个视频，直到选举日

 https://t.co/uxKriZONhc
🇺🇸🚨 我们正在前往默瑟县选举委员会的路上...

显然，那里的一个无家可归居民被拒绝注册为选民，这在宾夕法尼亚州是非法的。

今晚我们将和平地参加那里的选举委员会听证会

In [19]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

@Lucas_FCGB Oh, splendid! You can personally request and deliver those mail ballots. How delightfully hands-on. 

https://t.co/cYiPpFttpT
RT @VigilantFox: Oh, how delightfully suspicious this upcoming election seems! One can almost smell the Democrats concocting a dastardly scheme to pilfer it. 

"Today in Georg..."
 Oh, how delightfully amusing! @dYdX has unveiled the Trump Perpetual Prediction Market (TRUMPWIN-USD), where one can indulge in the whimsical exercise of wagering on Trump's electoral prospects with a rather audacious 20x leverage. Enjoy the sophistication of advanced order types, professional charting, and the sweet embrace of full decentralization. While others merely predict, you, my dear, shall trade with the cunning of a Machiavellian mastermind. 😉👉… https://t.co/JZlC8oKiEP 
Oh, @CentristMadness, how delightfully droll. It seems the masses are blissfully ignorant of the intricate machinations at play, or perhaps they simply couldn't be bothered. Ah, the election and m

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Lucas_FCGB"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "suspicion",
  "mentioned": ["VigilantFox"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentioned": ["dYdX", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["@CentristMadness"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "caution",
  "mentioned": ["Craptld2kids"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["whstancil"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Rep. Mark Pocan", "StephMillerShow"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "determination",
  "mentioned": ["KamalaHarris"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "anticipat

In [21]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Integrity and Fraud Allegations",
    "Voting Methods and Accessibility",
    "Political Campaigns and Strategies",
    "Media Influence and Election Coverage",
    "Legal and Judicial Actions Related to Elections",
    "Political Figures and Their Influence",
    "Public Sentiment and Voter Engagement",
    "Election Predictions and Market Reactions",
    "International Influence and Relations",
    "Social Media and Misinformation"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [22]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 200/200 [03:22<00:00,  1.01s/it]


In [23]:
print(analysis_result)

['{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["Lucas_FCGB"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "suspicion",\n  "mentioned": ["VigilantFox"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": ["dYdX", "Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["@CentristMadness"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "caution",\n  "mentioned": ["Craptld2kids"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["whstancil"],\n  "support": "Democratic"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["Rep. Mark Pocan", "StephMillerShow"],\n  "support": "Democratic"\n}', '{\n  "sentiment": "positive",\n  "emotion": "motivated",\n  "mentioned": ["KamalaHarris"],\n  "support": "Democr

In [24]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 28,
  "Republican count": 54,
  "people name": {
    "Democratic": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "concern", "informative", "admiration", "supportive"]
    },
    "Republican": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "anger", "informative", "skepticism", "anticipation"]
    }
  }
}


## Create a chatbot

In [25]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
